In [2]:
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
import PySimpleGUI as sg
import yaml

In [13]:
w = 200 * 3
h = 20
d_w = 20.0
s = 0

c = None
txt = None

layout = [
    [sg.Graph((w, h), (0,0), (w, h),  background_color='lime', key='-P-')],
    [sg.Button('+', key='-A-')],
    [sg.Button('S', key='-S-'), [sg.Button('E', key='-E-')],],
]

window = sg.Window('App', layout, finalize=True)


def draw_rect(s):
    c1 = s * d_w
    s += 1
    c2 = s * d_w
    
    window['-P-'].draw_rectangle(
        (c1, 0),
        (c2, h),
        fill_color = c
    )
    return s

  
def draw_txt(s):
    x = (s - 1) * d_w + d_w / 2
    
    window['-P-'].draw_text(
        txt,
        (x, h / 2),
    )


while True:
    event, values = window.read()
    
    if event == sg.WIN_CLOSED:
        break
    
    if event == '-A-':
        s = draw_rect(s)
        draw_txt(s)
        
    if event == '-S-':
        c = 'orange'
        txt = 'S'
        
    if event == '-E-':
        c = 'red'
        txt = 'E'
        
    window.read(timeout=0)
        
window.close()

In [12]:
%pip install pyyaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.8/661.8 kB 5.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


# kéne egy olyan funkció, amivel sub actionöket lehet csinálni

In [19]:
def load_config():
    with open(Path.cwd().joinpath('config.yaml'), 'r') as f:
        y = yaml.load(f, yaml.FullLoader)
    return y

def get_timestep(cap):
    return int(cap.get(cv2.CAP_PROP_POS_FRAMES))


def update_duration(window, cap):
    window['-DURATION-'].update(
        f'{round(cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS), 3)} s'
    )


def encode_as_bytes(f, w, h):
    f = cv2.resize(f, (w, h))
    return cv2.imencode('.ppm', f)[1].tobytes()


def update_seconds(window, cap):
    timestep = cap.get(cv2.CAP_PROP_POS_FRAMES)
    fps = cap.get(cv2.CAP_PROP_FPS)
    seconds = round(timestep / fps, 3)
    window['-SECONDS-'].update(f'{seconds} s')

   
def update_image(window, cap, w, h):
    ret, f = cap.read()
    if not ret:
        sg.popup('No more frames left!')
        return
    fbytes = encode_as_bytes(f, w, h)
    window['-IMAGE-'].update(data=fbytes)
    
    
def update_play_button(window, is_playing):
    window['-PLAY-'].update('Pause' if is_playing else 'Play')


def load_video(p, config):
    cap = cv2.VideoCapture(str(p))
    return cap
    
    
def load_from_beginning(window, cap, w, h):
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    ret, f = cap.read()
    fbytes = encode_as_bytes(f, w, h)
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    window['-IMAGE-'].update(data=fbytes)
    

def load_prev(window, cap, w, h):
    t = cap.get(cv2.CAP_PROP_POS_FRAMES)
    cap.set(cv2.CAP_PROP_POS_FRAMES, t - 2)
    ret, f = cap.read()
    fbytes = encode_as_bytes(f, w, h)
    window['-IMAGE-'].update(data=fbytes)

   
def load_next(window, cap, w, h):
    ret, f = cap.read()
    fbytes = encode_as_bytes(f, w, h)
    window['-IMAGE-'].update(data=fbytes)

    
def draw_timestep_on_progressbar(window, cap, unit, h, txt, color):
    timestep = get_timestep(cap)
            
    x1 = (timestep - 1) * unit
    y1 = h
    x2 = timestep * unit
    y2 = 0
    
    drawing_id = window['-PROGRESS-'].draw_rectangle(
        (x1, y1),
        (x2, y2),
        fill_color=color,
        line_color = color,
        line_width=2,
    )
    
    return drawing_id
    
    
def update_table(window, tvalues):
    window['-TABLE-'].update(tvalues)
    
    
def update_counter(window, cap):
    window['-COUNTER-'].update(get_timestep(cap))
    
    
def export_to_csv(window, video, rows):
    out_path = Path.cwd().joinpath('output', f'labels_for_{video}.csv')
    df = pd.DataFrame(rows, columns=['Start', 'End', 'Action'])
    df['Action'] = df['Action'].map(lambda x: '_'.join(str(x).lower().split(' ')))
    df.to_csv(out_path, index=False)
    sg.popup(f'Saved csv as {out_path.name} to {out_path.parent}')
    
    
def get_center_coordinates_on_graph(label, graph_step_size):
    start = label[0]
    end = label[1]
    
    s_x = start * graph_step_size - graph_step_size / 2
    e_x = end * graph_step_size - graph_step_size / 2
    
    return (s_x, e_x) 
    

def app():
    img_width = 500
    img_height = 500
    
    graph_width = img_width * 2
    graph_height = 30
    graph_stride = None
    
    cap = None
    is_playing = False
    
    labels = []
    headings = ['Start', 'End', 'Action']
    
    config = load_config()
    
    start = None
    end = None
    action = None
    
    video = None
    
    timestep = 0
    
    mleft = [
        [sg.Image(size=(img_width, img_height), background_color='white', key='-IMAGE-')],
        [sg.FileBrowse('Load video', file_types=(("MP4 files", "*.mp4"),), key='-LOAD-', enable_events=True)],
        [sg.Text('Time:'), sg.Text(key='-SECONDS-'), sg.Text('/'), sg.Text(key='-DURATION-'), sg.Text('Frame:'), sg.Text(key='-COUNTER-')],
    ]
    
    sleft = [
        [sg.Table(labels, headings, num_rows=35, col_widths=[6, 6, 25], expand_x=True, select_mode=sg.TABLE_SELECT_MODE_BROWSE, key='-TABLE-', enable_events=True)],
        [sg.Button('Export', expand_x=True, key='-EXPORT-')],
    ]
    
    sright = [
        [sg.Button('Play', expand_x=True, key='-PLAY-'), sg.Button('Restart', expand_x=True, key='-RESTART-')],
        [sg.Button('S', expand_x=True, key='-S-'), sg.Button('E', expand_x=True, key='-E-')],
        [sg.Button('<', expand_x=True, key='-PREV-'), sg.Button('>', expand_x=True, key='-NEXT-')],
        [sg.Text('What action did you see?')],
        [sg.Combo(config['classes'], key='-COMBO-', expand_x=True, enable_events=True)],
        [sg.Button('+', expand_x=True, key='-ADD-'), sg.Button('-', expand_x=True, key='-DELETE-')],
        [sg.Button('debug', key='-DEBUG-')],
    ]
    
    mright = [
        [sg.Text('Currently playing:', expand_x=True), sg.Text('', key='-VIDEO-')],
        [sg.Column(sleft, expand_x=True), sg.Column(sright, vertical_alignment='top')]
    ]
    
    layout = [
        [sg.Column(mleft), sg.Column(mright, expand_x=True, expand_y=True)],
        [sg.Graph((graph_width, graph_height), (0,0), (graph_width, graph_height),  background_color='lime', key='-PROGRESS-')],
    ]

    window = sg.Window('Video Labeling Tool 2023 by Semsey Dániel', layout, finalize=True)

    while True:
        event, values = window.read(timeout=0)
        
        if event == sg.WIN_CLOSED:
            break
        
        if event == '-LOAD-':
            video_path = Path(values['-LOAD-'])
            video = video_path.stem
            cap = load_video(video_path, config)
            window['-SECONDS-'].update(f'0 s')
            window['-VIDEO-'].update(video_path.name)
            load_from_beginning(window, cap, img_width, img_height)
            update_duration(window, cap)
            update_counter(window, cap)
            graph_stride = int(graph_width / cap.get(cv2.CAP_PROP_FRAME_COUNT))
            
        if event == '-PLAY-' and video:
            is_playing = not is_playing
            update_play_button(window, is_playing)
            
        if event == '-RESTART-':
            load_from_beginning(window, cap, img_width, img_height)
            update_seconds(window, cap)
            update_counter(window, cap)
            is_playing = False
            update_play_button(window, is_playing)
            
        if event == '-PREV-':
            if get_timestep(cap) < 1:
                sg.popup('Invalid step!')
            else:
                load_prev(window, cap, img_width, img_height)
                update_seconds(window, cap)
                update_counter(window, cap)
                is_playing = False
                update_play_button(window, is_playing)
            
        if event == '-NEXT-':
            if get_timestep(cap) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
                sg.popup('Invalid step!')
            else:
                load_next(window, cap, img_width, img_height)
                update_seconds(window, cap)
                update_counter(window, cap)
                is_playing = False
                update_play_button(window, is_playing)
            
        if event == '-S-':
            if start == None:
                temp = get_timestep(cap)
                if end != None:
                    if temp < end:
                        start = temp
                        draw_timestep_on_progressbar(window, cap, graph_stride, graph_height, 'S', config['start color'])
                    else:
                        sg.popup('Start must happen before end!')
                else:
                    start = temp
                    draw_timestep_on_progressbar(window, cap, graph_stride, graph_height, 'S', config['start color'])
            else:
                sg.popup('Finish labeling the current action!')
            
        # End cannot start before start            
        if event == '-E-':
            if end == None:    
                temp = get_timestep(cap)
                if start != None:
                    if start < temp:
                        end = temp
                        draw_timestep_on_progressbar(window, cap, graph_stride, graph_height, 'E', config['end color'])
                    else:
                        sg.popup('End must happen after start!')
                else:
                    sg.popup('You must set start first!')
            else:
                sg.popup('Finish labeling the current action!')
            
            
        if event == '-COMBO-':
            action = values['-COMBO-']
        
        # Add label to table
        if event == '-ADD-':
            # start and end can both be 0
            if start != None and end != None:
                # action is an empty string by default
                if action:
                    label = [start, end, action]
                    labels.append(label)
                    update_table(window, labels)
                    start = None
                    end = None
                else:
                    sg.popup('No label selected!')
            else:
                sg.popup('Wrong label format!')
        
        # Delete label from table
        # and from the progress bar    
        if event == '-DELETE-':
            selected_rows = values['-TABLE-']
            if selected_rows:
                selected_row = selected_rows[0]
                label = labels[selected_row]
                
                s_x, s_e = get_center_coordinates_on_graph(label, graph_stride)
                
                s_id = window['-PROGRESS-'].get_figures_at_location((s_x, graph_height / 2))
                e_id = window['-PROGRESS-'].get_figures_at_location((s_e, graph_height / 2))
                
                window['-PROGRESS-'].delete_figure(s_id)
                window['-PROGRESS-'].delete_figure(e_id)
                
                del labels[selected_row]
                update_table(window, labels)
            else:
                sg.popup('No labels to delete!')
                
        if event == '-EXPORT-':
            if labels:
                export_to_csv(window, video, labels)
            else:
                sg.popup('No labels to export!')
                
        if event == '-DEBUG-':
            sg.show_debugger_window()
        
        if is_playing:
            update_seconds(window, cap)
            update_image(window, cap, img_width, img_height)
            update_counter(window, cap)
            
        if cap and get_timestep(cap) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
            is_playing = False
            sg.popup('No more frames left!')
            
    window.close()
    
app()